In [67]:
from pathlib import Path
from glob import glob
import os
import pandas as pd
# import polars as pl
import plotly.express as px
from tqdm import tqdm

from abyss import dataparser as dp
from abyss.ingest_xls import (
    read_setitec_xls_to_series,
    process_setitec_xls_files_parallel,
)

mdb_dir=Path(r"C:\Users\NG9374C\Documents\GitHub\uos-drilling\abyss\sandbox\validation\113-V3-022-MQ_1")
#C:\Users\NG9374C\Documents\drilling_data\de0-svm42__AMIGO_ATIP_Digital_Assembly_WP3\RAW_SETITEC
ham_dir=Path(r"C:\Users\NG9374C\Documents\drilling_data\de0-svm42__AMIGO_ATIP_Digital_Assembly_WP3\RAW_SETITEC")
ham_selected_dir = ham_dir / ".." / "selected_control_boxes"

In [2]:
xls_files = glob(str(ham_selected_dir / "**/*.xls"), recursive=True)
len(xls_files)

3087

In [3]:
s = process_setitec_xls_files_parallel(xls_files, with_data=True, n_jobs=8)

Processing SETITEC XLS files in parallel:   1%|          | 16/3087 [00:02<09:27,  5.41it/s][Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    2.7s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    2.8s
Processing SETITEC XLS files in parallel:   1%|          | 24/3087 [00:02<05:37,  9.08it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    2.9s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    3.0s
Processing SETITEC XLS files in parallel:   1%|▏         | 40/3087 [00:03<02:44, 18.48it/s][Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    3.1s
[Parallel(n_jobs=8)]: Batch computation too fast (0.19901845683723546s.) Setting batch_size=2.
Processing SETITEC XLS files in parallel:   2%|▏         | 56/3087 [00:03<01:41, 29.92it/s][Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    3.2s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    3.3s
[Parallel(n_jobs=8)]: Batch computation too fast (0.19704747200012207s.) Setting batch_size=4.
Processing SETITEC 

In [47]:
l_keys_metadata = ['filename',
'directory',
'Date',
'Drilling Cycle ID',
'BOX Name',
'BOX SN',
'BOX Firmware Version',
'Motor Name',
'Motor SN',
'Head Global Counter',
'Head Local Counter 1',
]

l_keys_data = ['Position (mm)',
               'I Torque (A)', 'I Thrust (A)', 
               'I Torque Empty (A)', 'I Thrust Empty (A)', 
               'Step (nb)', 'Stop code', 'Torque Power (W)']

df = pd.DataFrame()

for item in tqdm(s, desc="Processing all dictionary entries"):
    if isinstance(item, dict):
        # If the item is a dictionary, extract metadata and data
        s_data = {key: item[key] for key in l_keys_data if key in item}
        df_data = pd.DataFrame(s_data)
        s_metadata = {key: item[key] for key in l_keys_metadata if key in item}
        df_metadata = pd.DataFrame(s_metadata, index=[0])
        # Concatenate metadata and data, repeating metadata for each row of data
        df_metadata = pd.concat([df_metadata] * len(df_data), ignore_index=True)
        df_item = pd.concat([df_metadata, df_data], axis=1)
        df = pd.concat([df, df_item], ignore_index=True)



Processing all dictionary entries:   9%|▉         | 293/3087 [01:56<18:32,  2.51it/s]


KeyboardInterrupt: 

## Converted dictionary entries into dataframes
We also do a datatype conversion to reduce sizes

In [ ]:
def process_item_to_dataframe(item):
    if isinstance(item, dict):
        try:
            # If the item is a dictionary, extract metadata and data
            s_data = {key: item[key] for key in l_keys_data if key in item}
            df_data = pd.DataFrame(s_data)
            s_metadata = {key: item[key] for key in l_keys_metadata if key in item}
            df_metadata = pd.DataFrame(s_metadata, index=[0])
            # Concatenate metadata and data, repeating metadata for each row of data
            df_metadata = pd.concat([df_metadata] * len(df_data), ignore_index=True)
            df_item = pd.concat([df_metadata, df_data], axis=1)
            # Ensure the DataFrame has a consistent index
            df_item.reset_index(drop=True, inplace=True)
            df_item = df_item.convert_dtypes()
            return df_item
        except Exception as e:
            print(f"Error processing item: {e}")
            return pd.DataFrame()
    return pd.DataFrame()

In [64]:
n_jobs = 8
from joblib import Parallel, delayed
list_df_parallel = Parallel(n_jobs=n_jobs, verbose=10)(
    delayed(process_item_to_dataframe)(item) for item in tqdm(s, desc="Creating SETITEC dataframes  in parallel")
)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    2.1s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    2.3s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    2.5s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.7s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    3.0s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    3.3s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    3.5s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    3.8s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:    4.1s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    4.4s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:    4.8s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    5.1s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:    5.6s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    5.9s
[Parallel(

In [65]:
df = pd.concat(list_df_parallel, ignore_index=True)
df.reset_index(drop=True, inplace=True)

In [66]:
df.tail()

,filename,directory,BOX Name,BOX SN,Motor SN,Head Global Counter,Position (mm),I Torque (A),I Thrust (A),I Torque Empty (A),Step (nb),Stop code,Date,Drilling Cycle ID,BOX Firmware Version,Motor Name,Head Local Counter 1,I Thrust Empty (A),Torque Power (W)
7490233,20110020_20110020_ST_2248_64,LEDU20100002,20100002,20100002,20110020,2248,-31.804001,5.811,0.34,5.029,4,0,2023-08-24:09:06:26,2873257258.0,V 3.2.2.1,Light_EDU ,64,0.328,227
7490234,20110020_20110020_ST_2248_64,LEDU20100002,20100002,20100002,20110020,2248,-31.827,5.908,0.48,5.029,4,0,2023-08-24:09:06:26,2873257258.0,V 3.2.2.1,Light_EDU ,64,0.328,230
7490235,20110020_20110020_ST_2248_64,LEDU20100002,20100002,20100002,20110020,2248,-31.85,6.006,0.703,5.029,4,0,2023-08-24:09:06:26,2873257258.0,V 3.2.2.1,Light_EDU ,64,0.328,232
7490236,20110020_20110020_ST_2248_64,LEDU20100002,20100002,20100002,20110020,2248,-31.85,6.006,0.703,5.029,4,8,2023-08-24:09:06:26,2873257258.0,V 3.2.2.1,Light_EDU ,64,0.328,232
7490237,20110020_20110020_ST_2248_64,LEDU20100002,20100002,20100002,20110020,2248,-31.85,6.006,0.703,5.029,4,8,2023-08-24:09:06:26,2873257258.0,V 3.2.2.1,Light_EDU ,64,0.328,232


### Corrected the types of several columns
Otherwise this cannot be saved in a parquet

In [82]:
df1 = df.astype({
           "BOX Name": "string",
           "BOX SN": "string",
           "BOX Firmware Version": "string",
          })
# df.to_parquet(ham_selected_dir / "HAM_selected_setitec_data.parquet", index=False)

In [84]:
# s[-1].keys()
# s_metadata = {key: s[-1][key] for key in l_keys_metadata}
# df.head()
# process_item_to_dataframe(s[-1])
df1.to_parquet(ham_selected_dir / "HAM_selected_setitec_data.parquet", index=False, compression="brotli")

## Pickling and saving the dictionary

In [86]:
import pickle
pickle_file = ham_selected_dir / "HAM_unlabelled_signal_dict_20250617.pkl"

with open(pickle_file, 'wb') as f:
    pickle.dump(s, f)

import json
json_file = ham_selected_dir / "HAM_unlabelled_signal_dict_20250617.json"
with open(json_file, 'w') as f:
    json.dump(s, f, indent=4, ensure_ascii=False)